In [ ]:
# Import modules
import torch
import os

import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np

from torch import nn, optim
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import models
from torch.cuda import is_available

In [ ]:
device=torch.device('cuda' if is_available() else 'cpu')
device

In [ ]:
# Define tranform function
tranform=transforms.Compose([transforms.ToTensor(), # Convert image to tensor
                             transforms.Normalize((0.1307,),(0.3081,))]) # Normalize tensor
train_data=MNIST(root='./train',train=True,transform=tranform,download=True) # Download train data
test_data=MNIST(root='./test',train=False,transform=tranform,download=True) # Download test data

print('Train Data Length:',len(train_data))
print('Test Data Length:',len(test_data))

In [ ]:
# ResNet Based Model
class Model(nn.Module):
    def __init__(self,num_classes):
        super(Model,self).__init__()
        # As ResNet designed for 3-dimensional RGB image, Strech 1-dimensional MNIST image to 3-dimensional tensor
        self.conv2d=nn.Conv2d(1,3,5)
        # Backbone models was trained with ImageNet datasets
        self.backbone=models.resnet18(pretrained=True)
        self.backbone.fc=nn.Linear(self.backbone.fc.in_features,num_classes)

    def forward(self,x):
        x=self.conv2d(x)
        x=self.backbone(x)
        return x

In [ ]:
model=Model(10)
model=model.to(device)

criterion=nn.CrossEntropyLoss(reduction='mean')
criterion=criterion.to(device)

optimizer=optim.Adam(model.parameters(),lr=0.001)
print(model)

In [ ]:
batch_size=256

# Dataloader batches dataset for batch_size
train_loader=DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_data,batch_size=batch_size,shuffle=False)

In [ ]:
n_epochs=5

for epoch in range(n_epochs):
    # Train Step
    model.train()
    train_losses=[]
    for i,(data,target) in enumerate(train_loader):
        data,target=data.to(device),target.to(device)
        # Forward operation
        pred=model(data)
        # Calculate loss
        loss=criterion(pred,target)
        # Calculate gradients
        loss.backward()
        # Update weights
        optimizer.step()
        # Initialize gradients
        optimizer.zero_grad()
        train_losses.append(loss.item())
        if i % 100 == 0:
            print('Train Epoch: {}({:.4f}%) Loss:{:.8f}'.format(epoch,(i+1)/len(train_loader)*100,np.mean(train_losses)))

In [ ]:
# Test step
model.eval()
correct=0
for data,label in test_loader:
    data,label=data.to(device),label.to(device)
    # Forward operation
    pred=model(data)
    pred_max=np.argmax(pred.detach().cpu(),axis=1)
    for pair in zip(label,pred_max):
        print('Label: {}, Pred: {}'.format(pair[0],pair[1]))
        if pair[0]==pair[1]:
            correct+=1

print('Accuracy: {:.2f}%'.format(correct/len(test_data)*100))

In [ ]:
# Upload your own image and test
from google.colab import files
from PIL import Image, ImageOps
from IPython.display import display

custom_transform=transforms.Compose([transforms.Resize((28,28)),transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])
uploaded=files.upload()
filename=next(iter(uploaded))
img=Image.open(filename)
img=img.convert('1')
img_reverse=ImageOps.invert(img.convert('RGB')).convert('1')
img,img_reverse=custom_transform(img),custom_transform(img_reverse)

model.eval()
ToPILConverter=transforms.ToPILImage()
for i in [img,img_reverse]:
    pred=model(i.unsqueeze(0).to(device))
    pred_max=np.argmax(pred.detach().cpu(),axis=1)
    i=ToPILConverter(i)
    display(i)
    print('PRED: {}'.format(pred_max.item()))